In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
with open('/kaggle/input/simplified-macbeth/simplified_macbeth', 'r') as f:
    text = f.read()

lines = text.split('\n\n')
without_name = [''.join(l.split(':')[1:])[1:] for l in lines]
originals = [l.split('[[')[0][:-1] for l in without_name]
translated = [l.split('[[')[1].split(']]')[0] for l in without_name]
print(originals[115], '---->', translated[115])

import pandas as pd
from datasets import Dataset

data = {"input": originals, "output": translated}
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

We will proceed no further in this business He hath honour'd me of late; and I have bought Golden opinions from all sorts of people, Which would be worn now in their newest gloss, Not cast aside so soon. ----> We will not continue with this plan. He has honored me recently, and I have earned the respect of all kinds of people, which I want to enjoy now, not throw away so soon.


In [5]:
prompt = """Below is an instruction on how to translate text from complex and archaic to simple and modern english.
The input contains this text. Write in the response a faithful translation.

### Instruction:
Translate the following line from complex to simple english.
It may contain archaic and literary words or structures: make them easily understandable.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(data):
    inputs = data['input']
    outputs = data['output']
    texts = []
    for inp, outp in zip(inputs, outputs):
        text = prompt.format(inp, outp) + EOS_TOKEN
        texts.append(text)
    return {"text" : texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/654 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,#num_train_epochs=1
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        report_to="none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/654 [00:00<?, ? examples/s]

In [7]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.117 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 654 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.883800
2,2.868200
3,2.634300
4,2.675200
5,2.423500
6,1.974300
7,1.665000
8,1.437400
9,1.245000
10,0.977500


In [9]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

11.98 minutes used for training.
Peak reserved memory = 7.373 GB.
Peak reserved memory for training = 0.256 GB.
Peak reserved memory % of max memory = 50.017 %.
Peak reserved memory for training % of max memory = 1.737 %.


In [12]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "This above all: to thine own self be true, And it must follow, as the night the day,Thou canst not then be false to any man.",
        "",
    )
], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=2048, use_cache=True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction on how to translate text from complex and archaic to simple and modern english.\nThe input contains this text. Write in the response a faithful translation.\n\n### Instruction:\nTranslate the following line from complex to simple english.\nIt may contain archaic and literary words or structures: make them easily understandable.\n\n### Input:\nThis above all: to thine own self be true, And it must follow, as the night the day,Thou canst not then be false to any man.\n\n### Response:\nAbove all else, be true to yourself, and it will naturally follow that you cannot be false to anyone else.<|end_of_text|>']